In [6]:
from fairchem.core import FAIRChemCalculator
import ase.io
from ase.optimize import LBFGS

from fairchem.data.oc.core import Adsorbate, AdsorbateSlabConfig, Bulk, Slab
import os
from glob import glob
import pandas as pd
from fairchem.data.oc.utils.vasp import write_vasp_input_files

## Define desired adsorbate+slab system



In [8]:
bulk_src_id = "mp-30"
adsorbate_smiles = "*CO"

bulk = Bulk(bulk_src_id_from_db = bulk_src_id)
adsorbate = Adsorbate(adsorbate_smiles_from_db=adsorbate_smiles)
slabs = Slab.from_bulk_get_specific_millers(bulk = bulk, specific_millers=(1,1,1))

# There may be multiple slabs with this miller index.
# For demonstrative purposes we will take the first entry.
slab = slabs[0]

# AdsorbML

In [24]:
calc = FAIRChemCalculator(hf_hub_filename="uma_sm.pt", device="cuda", task_name="oc20")
adsorption_energy_model = False

### Automated

More details on the automated pipeline can be found at https://github.com/facebookresearch/fairchem/blob/main/src/fairchem/core/components/calculate/recipes/adsorbml.py#L316.

In [11]:
from fairchem.core.components.calculate.recipes.adsorbml import run_adsorbml

outputs = run_adsorbml(
    slab=slab,
    adsorbate=adsorbate,
    calculator=calc,
    optimizer_cls=LBFGS,
    fmax=0.02,
    steps=20,         # Increase to 200 for practical application, 20 is used for demonstrations
    num_placements=10, # Increase to 100 for practical application, 10 is used for demonstrations
    reference_ml_energies=not adsorption_energy_model, #True if using a total energy model (i.e. UMA)
    relaxed_slab_atoms=None,
    place_on_relaxed_slab=False,
)

       Step     Time          Energy          fmax
LBFGS:    0 15:21:19     -300.187740        0.048987
LBFGS:    1 15:21:19     -300.188282        0.046745
LBFGS:    2 15:21:19     -300.194195        0.005680
       Step     Time          Energy          fmax
LBFGS:    0 15:21:19     -333.950279        2.272333
LBFGS:    1 15:21:19     -334.014906        1.085944
LBFGS:    2 15:21:20     -334.051695        0.802282
LBFGS:    3 15:21:20     -334.168344        1.425189
LBFGS:    4 15:21:20     -334.201616        0.991624
LBFGS:    5 15:21:20     -334.307863        0.454191
LBFGS:    6 15:21:20     -334.329063        0.468656
LBFGS:    7 15:21:20     -334.365997        0.546173
LBFGS:    8 15:21:20     -334.407471        0.623766
LBFGS:    9 15:21:20     -334.437610        0.340172
LBFGS:   10 15:21:21     -334.449693        0.289551
LBFGS:   11 15:21:21     -334.460455        0.297633
LBFGS:   12 15:21:21     -334.474140        0.409112
LBFGS:   13 15:21:21     -334.485840        0.3001

In [18]:
top_candidates = outputs["adslabs"]
global_min_candidate = top_candidates[0]

In [47]:
top_candidates = outputs["adslabs"]
pd.DataFrame(top_candidates)

,input_atoms,atoms,results
0,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.6358430925115, 'forces': [[0.0..."
1,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.59077816636403, 'forces': [[0...."
2,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.58425503404, 'forces': [[0.0, ..."
3,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.5764463487371, 'forces': [[0.0..."
4,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.56220799119313, 'forces': [[0...."
5,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.5186574998601, 'forces': [[0.0..."
6,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.4857214036687, 'forces': [[0.0..."
7,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.4530485215888, 'forces': [[0.0..."
8,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.4447954240544, 'forces': [[0.0..."
9,"{'atoms': (Atom('Cu', [np.float64(1.3000465215...","(Atom('Cu', [np.float64(1.3000465215529724), n...","{'energy': -334.29819470078786, 'forces': [[0...."


# Write VASP input files

This assumes you have access to VASP pseudopotentials. The default VASP flags (which are equivalent to those used to make OC20) are located in `ocdata.utils.vasp`. Alternatively, you may pass your own vasp flags to the `write_vasp_input_files` function as `vasp_flags`

In [46]:
# Grab the 5 systems with the lowest energy
top_5_candidates = top_candidates[:5]

# Write the inputs
for idx, config in enumerate(top_5_candidates):
    os.makedirs(f"data/{idx}", exist_ok=True)
    write_vasp_input_files(config["atoms"], outdir = f"data/{idx}/")